In [54]:
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os
import requests
import alpaca_trade_api as tradeapi
import matplotlib.pyplot as plt
import hvplot.pandas
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [55]:
load_dotenv()

True

In [56]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

In [57]:
 # Verify that Alpaca key and secret were correctly loaded
print(f"Alpaca Key type: {type(alpaca_api_key)}")
print(f"Alpaca Secret Key type: {type(alpaca_secret_key)}")

Alpaca Key type: <class 'str'>
Alpaca Secret Key type: <class 'str'>


In [58]:
 # Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

In [59]:
# Format current date as ISO format
start = pd.Timestamp("2019-06-01", tz="America/New_York").isoformat()
end = pd.Timestamp("2019-12-30", tz="America/New_York").isoformat()

In [60]:
 # Set the tickers
tickers = ["AMD", "TSLA"]

In [61]:
timeframe = "1D"

In [62]:
# Get current closing prices for FB and TWTR
df_portfolio = alpaca.get_barset(
    tickers,
    timeframe,
    start = start,
    end = end
).df

# Display sample data
df_portfolio

AMD                                       TSLA  \
                            open    high     low   close     volume    open   
time                                                                          
2019-06-03 00:00:00-04:00  28.75  29.620  27.290  27.575  138199999  185.51   
2019-06-04 00:00:00-04:00  28.39  29.590  27.910  29.565   99413074  181.10   
2019-06-05 00:00:00-04:00  29.93  29.990  29.030  29.500   73327828  198.47   
2019-06-06 00:00:00-04:00  29.87  31.980  29.840  31.840  121837973  204.25   
2019-06-07 00:00:00-04:00  31.94  32.600  31.570  32.410   78557712  205.00   
...                          ...     ...     ...     ...        ...     ...   
2019-12-23 00:00:00-05:00  44.58  45.625  44.390  45.470   52586935  411.78   
2019-12-24 00:00:00-05:00  46.10  46.610  45.770  46.530   42418841  418.84   
2019-12-26 00:00:00-05:00  46.99  47.310  45.660  46.640   55021416  427.91   
2019-12-27 00:00:00-05:00  46.84  46.880  45.975  46.180   35145491  435.25   
2019-12-30 00:00:00-05:00  46.14  46.170  44.660  45.520   40046758  428.81   

                                                                   
                               high       low     close    volume  
time                                                               
2019-06-03 00:00:00-04:00  186.6800  176.9919  179.0100  12380263  
2019-06-04 00:00:00-04:00  193.9800  179.6100  193.9100  13115692  
2019-06-05 00:00:00-04:00  201.2782  191.8486  196.6770  12570739  
2019-06-06 00:00:00-04:00  211.0000  201.8000  206.0800  19472101  
2019-06-07 00:00:00-04:00  210.8438  203.4990  204.3900  14897026  
...                             ...       ...       ...       ...  
2019-12-23 00:00:00-05:00  422.0100  410.2700  419.1800  12621840  
2019-12-24 00:00:00-05:00  425.4700  412.6875  425.1800   7833308  
2019-12-26 00:00:00-05:00  433.4800  426.3500  430.8202  10131052  
2019-12-27 00:00:00-05:00  435.3100  426.1100  430.2300   9209880  
2019-12-30 00:00:00-05:00  429.0000  409.2589  414.6200  12082466  

[147 rows x 10 columns]

In [63]:
# Create and empty DataFrame for closing prices
amd_df = pd.DataFrame()

# Fetch the closing prices of FB and TWTR
amd_df["AMD Close"] = df_portfolio["AMD"]["close"]
amd_df["AMD Volume"] = df_portfolio["AMD"]["volume"]
#df_closing_prices["TSLA"] = df_portfolio["TSLA"]["close"]
#df_closing_prices["TSLA Volume"] = df_portfolio["TSLA"]["volume"]

# Drop the time component of the date
amd_df.index = amd_df.index.date

# Display sample data
amd_df.head(10)

,AMD Close,AMD Volume
2019-06-03,27.575,138199999
2019-06-04,29.565,99413074
2019-06-05,29.500,73327828
2019-06-06,31.840,121837973
2019-06-07,32.410,78557712
2019-06-10,33.230,89542352
2019-06-11,32.410,76741730
2019-06-12,32.180,52750156
2019-06-13,31.380,115490630
2019-06-14,30.350,130024023


In [64]:
amd_options_df = pd.read_csv("Resources/amd_volume.csv")

#set index to date
amd_options_df.DataDate = pd.to_datetime(amd_options_df.DataDate, infer_datetime_format=True)
amd_options_df = amd_options_df.set_index('DataDate')

In [65]:
combined = amd_df.merge(amd_options_df,  left_index =True, right_index =True, how = 'inner')
combined.rename(columns ={"Percent of cPut": "Percent of Put", "Total Volume": "Total Option Volume"}, inplace = True)
combined.head()

,AMD Close,AMD Volume,Call Volume,Put Volume,Total Option Volume,Percent of Put,Put signals,Percent of calls,Call signals
2019-06-06,31.84,121837973,483512.0,261345.0,744857.0,0.350866,False,0.649134,True
2019-06-07,32.41,78557712,289447.0,155180.0,444627.0,0.349012,False,0.650988,True
2019-06-10,33.23,89542352,378452.0,175315.0,553767.0,0.316586,False,0.683414,True
2019-06-11,32.41,76741730,214939.0,142684.0,357623.0,0.398979,False,0.601021,True
2019-06-12,32.18,52750156,141541.0,82821.0,224362.0,0.369140,False,0.630860,True


In [66]:
#construct bollinger band and append to amd_df
bollinger_window = 20

#calculate rolling mean and SD
combined['bollinger_mid_band'] = combined['AMD Close'].rolling(window=bollinger_window).mean()
combined['bollinger_std'] = combined['AMD Close'].rolling(window=20).std()

#Calculate upper and lowers bands of bollinger band
combined['bollinger_upper_band']  = combined['bollinger_mid_band'] + (combined['bollinger_std'] * 1)
combined['bollinger_lower_band']  = combined['bollinger_mid_band'] - (combined['bollinger_std'] * 1)

#Calculate bollinger band trading signal
combined['bollinger_long'] = np.where(combined['AMD Close'] < combined['bollinger_lower_band'], 1.0, 0.0)
combined['bollinger_short'] = np.where(combined['AMD Close'] > combined['bollinger_upper_band'], -1.0, 0.0)
combined['bollinger_signal'] = combined['bollinger_long'] + combined['bollinger_short']


In [67]:
#construct relative volume ratio and append to amd_df
volume_ratio = 20

#calculate rvol and 1 deviation of a 20 day rvol
combined['rvol'] = combined['AMD Volume']/ combined['AMD Volume'].rolling(window = volume_ratio).mean()
combined['rvol SD'] = combined['rvol'].rolling(window = volume_ratio).std()/2+ combined['rvol'].rolling(window = volume_ratio).mean()
#create rvol signal where rvol > 1 SD rvol
combined['rvol signal'] = np.where(combined['rvol'] > combined['rvol SD'], 1.0, 0.0)


In [68]:
#create options rVol signal
volume_ratio = 20

combined['Option rVol'] = combined['Total Option Volume']/ combined['Total Option Volume'].rolling(window = volume_ratio).mean()
combined['Option rVol SD'] = combined['rvol'].rolling(window = volume_ratio).std()/2+ combined['rvol'].rolling(window = volume_ratio).mean()
#create rvol signal where rvol > 1 SD rvol
combined['Option rVol signal'] = np.where(combined['rvol'] > combined['rvol SD'], 1.0, 0.0)

In [69]:
#create daily returns column and Actual Value column
combined['daily returns'] = combined['AMD Close'].dropna().pct_change()

In [71]:
#drop nans due to BB/rvol calculations
combined = combined.dropna()
combined

,AMD Close,AMD Volume,Call Volume,Put Volume,Total Option Volume,Percent of Put,Put signals,Percent of calls,Call signals,bollinger_mid_band,...,bollinger_long,bollinger_short,bollinger_signal,rvol,rvol SD,rvol signal,Option rVol,Option rVol SD,Option rVol signal,daily returns
2019-08-02,29.46,46470316,155666.0,133301.0,288967.0,0.461302,False,0.538698,False,32.94150,...,1.0,0.0,1.0,0.976222,1.053825,0.0,1.000210,1.053825,0.0,-0.013396
2019-08-05,27.98,67551620,177393.0,225181.0,402574.0,0.559353,False,0.440647,False,32.76600,...,1.0,0.0,1.0,1.367475,1.095401,1.0,1.355245,1.095401,1.0,-0.050238
2019-08-06,28.85,52564104,139467.0,77573.0,217040.0,0.357413,False,0.642587,True,32.60600,...,1.0,0.0,1.0,1.055476,1.108734,0.0,0.733816,1.108734,0.0,0.031094
2019-08-07,29.19,51792473,117435.0,72452.0,189887.0,0.381553,False,0.618447,True,32.40850,...,1.0,0.0,1.0,1.045626,1.114328,0.0,0.663929,1.114328,0.0,0.011785
2019-08-08,33.95,154602514,677838.0,295952.0,973790.0,0.303918,False,0.696082,True,32.41675,...,0.0,0.0,0.0,2.831795,1.329637,1.0,3.086563,1.329637,1.0,0.163070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-23,45.47,52586935,264480.0,131907.0,396387.0,0.332773,False,0.667227,True,40.76620,...,0.0,-1.0,-1.0,1.267045,1.014151,1.0,1.471474,1.014151,1.0,0.029665
2019-12-24,46.53,42418841,279346.0,128062.0,407408.0,0.314334,False,0.685666,True,41.10270,...,0.0,-1.0,-1.0,1.024291,1.027352,0.0,1.477695,1.027352,0.0,0.023312
2019-12-26,46.64,55021416,344651.0,178378.0,523029.0,0.341048,False,0.658952,True,41.48620,...,0.0,-1.0,-1.0,1.306320,1.063301,1.0,1.801463,1.063301,1.0,0.002364
2019-12-27,46.18,35145491,191984.0,104085.0,296069.0,0.351557,False,0.648443,True,41.82470,...,0.0,-1.0,-1.0,0.831136,1.073322,0.0,0.998239,1.073322,0.0,-0.009863


In [72]:
combined.to_pickle(r'C:\Users\Kiel\Desktop\FINTECH\UCB_fintech_homework\project_2\Resources\amd_features_df.pickle')
